# Workload example: Monte-Carlo estimate of $\pi$

<img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" width="25%" align=left alt="PI monte-carlo estimate"/>

Let's assume we would like to estimate the number $\pi$ using a [Monte-Carlo method](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods), i.e. obtain an estimate numerically based on a random sampling approach.

We take advantage of the geometrical fact that the area of a quarter circle with unit radius is $\pi/4$ and that the probability of randomly chosen points in a unit square to be inside the quarter circle is $\pi/4$ as well. So, for $N_{total}$ randomly chosen pairs $(x, y)$ with $x\in[0, 1)$ and $y\in[0, 1)$ we count the number $N_{circ}$ of pairs that satisfy $(x^2 + y^2) < 1$ and estimate $\pi \approx 4 \cdot N_{circ} / N_{total}$.

In [1]:
from pi_workload import define_pi_workload

In [2]:
define_pi_workload??

Signature: define_pi_workload(size_in_terabytes=0.25, chunk_size_in_megabytes=500)
Source:   
def define_pi_workload(size_in_terabytes=0.25, chunk_size_in_megabytes=500):
    """Calculate pi using a Monte Carlo method."""
    
    print("workload in giga bytes:", size_in_terabytes*1e12/1e9)
    print(f"{(size_in_terabytes*1e12)/(chunk_size_in_megabytes*1e6)} chunks to process")

    total_array_size = (int(size_in_terabytes * 1e12 / 8 / 2), 2)
    number_of_tasks = (size_in_terabytes * 1e12) / (chunk_size_in_megabytes * 1e6)
    array_chunk_size = (int(total_array_size[0] / number_of_tasks), 2)

    xy = dask.array.random.uniform(
        low=0.0, high=1.0,
        size=total_array_size,
        chunks=array_chunk_size
    )

    xy_inside_circle = (xy ** 2).sum(axis=1) < 1

    pi = 4 * xy_inside_circle.mean()

    return pi
File:      ~/Course-Data-Science-with-Dask/04_distributed/pi_workload.py
Type:      function


## Default workload characteristics

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          125Gi       903Mi       122Gi        33Mi       2.7Gi       123Gi
Swap:         8.0Gi          0B       8.0Gi


In [4]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              16
On-line CPU(s) list: 0-15
Thread(s) per core:  1
Core(s) per socket:  1
Socket(s):           16
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 6148 CPU @ 2.40GHz
Stepping:            0
CPU MHz:             2394.375
BogoMIPS:            4788.75
Hypervisor vendor:   VMware
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            28160K
NUMA node0 CPU(s):   0-15
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon nopl xtopology tsc_reliable nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor l

For performance reasons, i.e. to avoid excessive disk spilling, Dask array chunks should fit several times per utilized CPU into a Dask worker's memory. For the above machine a value below

In [5]:
machine_mem = 125 # GiB
machine_cpus = 16
this_is_several_times = 10
conversion_factor = 2**30 / 10**6
print(f"{(machine_mem/machine_cpus)/this_is_several_times*conversion_factor} MB")

838.8607999999999 MB


should be sufficient.